In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Set hyperparameters
num_epochs = 3
batch_size = 100
learning_rate = 0.001

# Load MNIST dataset and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Define CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) #안 돌아갈 경우 수를 작게 64를 작게
        self.fc1 = nn.Linear(64 * 7 * 7, 128) #y*7*7
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7) #64채널개수 7*7높이너비
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = SimpleCNN()

#NN SVG인터넷 검색 후 사이트에서 실제로 볼 수 있음

# Model training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images) # forward-propagation
        loss = criterion(outputs, labels)
    
    optimizer.zero_grad() # back-propagation
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], '
            f'Step [{i+1}/{total_step}], '
            f'Loss: {loss.item():.4f}')
        
# Model testing
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy: {100 * correct / total}%')

Epoch [1/3], Step [600/600], Loss: 2.3014
Epoch [2/3], Step [600/600], Loss: 2.2543
Epoch [3/3], Step [600/600], Loss: 2.1575
Accuracy: 18.27%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformation
transform = transforms.Compose([
    transforms.Resize(224), # ResNet18 is built for 224 x 224 size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset MNIST 같은 이미지 데이터 클래스 10개만 추려놓음
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2)

# Load ResNet-18 and modify it
model = resnet18(weights=ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10) # CIFAR-10 has 10 classes 10개 짜리 추가해줌
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define training function
def train(epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 200 == 199:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
                running_loss = 0.0
                
# Define testing function
def evaluate():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on 10000 test images: {100 * correct / total:.2f}%')
    
# Model training with 5 epochs
train(5)
# 5바퀴정도만 진행을 해도 잘 나옴

# Model testing
evaluate()

Files already downloaded and verified
Files already downloaded and verified
[1,   200] loss: 0.980


In [29]:
import torch
import torch.nn as nn
import numpy as np

input_size = 4
hidden_size = 2

# 1-hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

input_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, e],
                       [l, e, e, l, l]], dtype=np.float32)

# Transform the numpy array into torch tensor
input_data = torch.tensor(input_data)
print(f'Input size: {input_data.shape}')

rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size)
outputs, _status = rnn(input_data)
print(f'Output size: {outputs.shape}')

Input size: torch.Size([3, 5, 4])
Output size: torch.Size([3, 5, 2])


In [30]:
import torch
import torch.nn as nn
import numpy as np

input_size = 4
hidden_size = 2

# 1-hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

input_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, e],
                       [l, e, e, l, l]], dtype=np.float32)

# Transform the numpy array into torch tensor
input_data = torch.tensor(input_data)
print(f'Input size: {input_data.shape}')

rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
outputs, _status = rnn(input_data)
print(f'Output size: {outputs.shape}')

Input size: torch.Size([3, 5, 4])
Output size: torch.Size([3, 5, 2])


In [31]:
import torch
import torch.nn as nn
import numpy as np

input_size = 4
hidden_size = 4

# 1-hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

input_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, e],
                       [l, e, e, l, l]], dtype=np.float32)

# Transform the numpy array into torch tensor
input_data = torch.tensor(input_data)
print(f'Input size: {input_data.shape}')

rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size)
outputs, _status = rnn(input_data)
print(f'Output size: {outputs.shape}')

Input size: torch.Size([3, 5, 4])
Output size: torch.Size([3, 5, 4])


In [32]:
import torch
import torch.nn as nn
# Define RNN
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        _, hidden = self.rnn(x)
        out = self.fc(hidden.squeeze(0))
        return out

In [33]:
# Set hyperparameters
input_size = 1
hidden_size = 10
output_size = 1
seq_length = 5
# Model initialization
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# Generate dataset
x = torch.randn(10, seq_length, input_size)
y = torch.sum(x, dim=1)

In [50]:
# Model training
for epoch in range(100):
    outputs = model(x)
    loss = criterion(outputs, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')
        
# Model testing
test_input = torch.randn(1, seq_length, input_size)
predicted = model(test_input)
print(f"Test input sum: {test_input.sum().item():.4f}")
print(f"Predicted sum: {predicted.item():.4f}")

Epoch [10/100], Loss: 0.0011
Epoch [20/100], Loss: 0.0006
Epoch [30/100], Loss: 0.0005
Epoch [40/100], Loss: 0.0005
Epoch [50/100], Loss: 0.0005
Epoch [60/100], Loss: 0.0004
Epoch [70/100], Loss: 0.0004
Epoch [80/100], Loss: 0.0004
Epoch [90/100], Loss: 0.0004
Epoch [100/100], Loss: 0.0003
Test input sum: -1.2030
Predicted sum: -0.6133


In [51]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)  # LSTM returns hidden state and cell state  
        out = self.fc(hidden.squeeze(0))
        return out

In [52]:
# Set hyperparameters
input_size = 1
hidden_size = 15
output_size = 1
seq_length = 5

# Model initialization
model = SimpleLSTM(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Generate dataset
x = torch.randn(10, seq_length, input_size)
y = torch.sum(x, dim=1)

# Model training
for epoch in range(100):
    outputs = model(x)
    loss = criterion(outputs, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')
# Model testing
test_input = torch.randn(1, seq_length, input_size)
predicted = model(test_input)
print(f"Test input sum: {test_input.sum().item():.4f}")
print(f"Predicted sum: {predicted.item():.4f}")

Epoch [10/100], Loss: 1.6402
Epoch [20/100], Loss: 1.1454
Epoch [30/100], Loss: 0.5434
Epoch [40/100], Loss: 0.1407
Epoch [50/100], Loss: 0.0671
Epoch [60/100], Loss: 0.0311
Epoch [70/100], Loss: 0.0132
Epoch [80/100], Loss: 0.0064
Epoch [90/100], Loss: 0.0030
Epoch [100/100], Loss: 0.0015
Test input sum: -1.6202
Predicted sum: -1.3167
